In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
!pip install keras_tuner
from keras_tuner.tuners import RandomSearch

print(tf.__version__)

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
2.7.0


In [2]:
# load train dataset
train_df = pd.read_csv('/home/featurize/data/training_norm.csv')
train_images = []
for image_id in train_df['image_id']:
    image = cv2.imread(f'/home/featurize/data/training_data/{image_id}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image = tf.image.resize(image, [60, 80])
    image = cv2.resize(image, (80, 60))
    #image = image / 255
    train_images.append(image)
y = []
x_names = []
# for rows in train_df:
#     print(rows)
    # f = rows[0]
    # a = float(rows[1])
    # s = float(rows[2])
    # y.append([a, s])
    # filename = str(f) + '.png'
    # x_names.append(filename)
train_images = np.array(train_images)
train_angles = np.array(train_df['angle'])
train_speeds = np.array(train_df['speed'])
print(np.array(train_df['angle']))
print(train_speeds)
print(train_angles)

# def preprocess(image):
#     im = tf.image.convert_image_dtype(image, tf.float32)
#     im = tf.image.resize(image, (160, 120))
#     im = tf.divide(im, 255)  # Normalize, need to check if needed
#     im = tf.expand_dims(im, axis=0) #add batch dimension
#     return image

# def img_preprocess(image):
#     #height, _, _ = image.shape
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space

#     image = cv2.resize(image, (120,160)) # input image size (200,66) Nvidia model
#     image = image / 255 # normalizing
#     return image

# load test dataset
test_images = []
for i in range(1, 1021):
    image = cv2.imread(f'/home/featurize/data/test_data/{i}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(image, [60, 80])
    test_images.append(image)
test_images = np.array(test_images)
test_ids = np.arange(1, 1021)

[0.4375 0.8125 0.4375 ... 0.5625 0.625  0.6875]
[0. 1. 1. ... 0. 0. 1.]
[0.4375 0.8125 0.4375 ... 0.5625 0.625  0.6875]


2023-05-06 22:47:55.412760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-06 22:47:55.420701: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-06 22:47:55.421151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-06 22:47:55.422805: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [3]:

# split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_df[['angle', 'speed']].values, test_size=0.2)
print(X_val.shape)
# define the early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
print(len(y_val))

(2759, 60, 80, 3)
2759


In [4]:
#Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = (60, 80)
IMG_SHAPE = IMG_SIZE + (3,)
print(IMG_SHAPE)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.summary()

(60, 80, 3)
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 80, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 30, 40, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 30, 40, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 30, 40, 32)   0           ['bn_Conv1[0][0]']               
                                                                   

In [5]:
# 数据增强操作
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])


In [6]:
# 数据进入MobileNetV2之前要先经过preprocess_input处理
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
# 全局平均池化层，参考： https://www.cnblogs.com/hutao722/p/10008581.html
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# 输出层
prediction_layer = tf.keras.layers.Dense(2)
# 输入
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
# 让MobileNetV2变得可训练
base_model.trainable = True
# 但是，前100层的参数让它冻结，我们只训练后面层的参数
fine_tune_at = 100
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


In [7]:
# 定义损失函数
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# 定义学习率
base_learning_rate = 0.00001
# 定义优化器
optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate)

model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['accuracy'])

/environment/miniconda3/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
# 开始训练 X_train, X_val, y_train, y_val
total_epochs = 200
# callback=keras.callbacks.EarlyStopping(monitor='loss',min_delta=0.002,patience=0,mode='auto',restore_best_weights=True)
history = model.fit( X_train, y_train, batch_size=64, epochs=total_epochs, verbose=1,
 validation_data=(X_val,y_val))
# history = model.fit(X_train,
#                     epochs=total_epochs,
#                     validation_data=X_val)


Epoch 1/200


2023-05-06 22:48:04.936056: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101


173/173 [==============================] - 11s 37ms/step - loss: 0.3523 - accuracy: 0.6702 - val_loss: 0.1034 - val_accuracy: 0.7847
Epoch 2/200
173/173 [==============================] - 5s 31ms/step - loss: 0.1137 - accuracy: 0.7702 - val_loss: 0.0756 - val_accuracy: 0.8358
Epoch 3/200
173/173 [==============================] - 6s 32ms/step - loss: 0.0934 - accuracy: 0.8022 - val_loss: 0.0672 - val_accuracy: 0.8456
Epoch 4/200
173/173 [==============================] - 5s 30ms/step - loss: 0.0826 - accuracy: 0.8285 - val_loss: 0.0607 - val_accuracy: 0.8637
Epoch 5/200
173/173 [==============================] - 6s 32ms/step - loss: 0.0752 - accuracy: 0.8374 - val_loss: 0.0550 - val_accuracy: 0.8818
Epoch 6/200
173/173 [==============================] - 5s 30ms/step - loss: 0.0695 - accuracy: 0.8576 - val_loss: 0.0554 - val_accuracy: 0.8800
Epoch 7/200
173/173 [==============================] - 5s 32ms/step - loss: 0.0653 - accuracy: 0.8659 - val_loss: 0.0490 - val_accuracy: 0.8927
Epo

In [9]:
# 在验证集里验证模型准确率
loss0, accuracy0 = model.evaluate(X_val)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

87/87 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
initial loss: 0.00
initial accuracy: 0.00


In [10]:
# 模型保存路径
MODEL_DIR = '/home/featurize/data'

# 保存模型
h5_dir = os.path.join(MODEL_DIR, 'mobilenet_v2_0506_2.h5')
model.save(h5_dir)

/environment/miniconda3/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [11]:

# 我们将保持好的模型再导入，再在验证集里查看它的准确率
new_model = tf.keras.models.load_model(h5_dir)
# make predictions on the test set
test_predictions = new_model.predict(test_images)

# get the steering angle and speed predictions separately
test_steering_predictions = test_predictions[:,0]
test_speed_predictions = test_predictions[:,1]
print(test_predictions)
print(test_steering_predictions)
print(test_speed_predictions)

submission_df = pd.DataFrame({
    'image_id': test_ids.flatten(),
    'angle': test_steering_predictions.flatten(),
    'speed': test_speed_predictions.flatten()
})
submission_df.to_csv('/home/featurize/data/submission-mobilenet0506_2.csv', index=False)

[[ 0.50357306 -0.00994746]
 [ 0.7336927   1.036348  ]
 [ 0.22143048  0.95613015]
 ...
 [ 0.4700193   0.99332124]
 [ 0.44930816  0.9749251 ]
 [ 0.31728747  0.9696951 ]]
[0.50357306 0.7336927  0.22143048 ... 0.4700193  0.44930816 0.31728747]
[-0.00994746  1.036348    0.95613015 ...  0.99332124  0.9749251
  0.9696951 ]


In [1]:
import tensorflow as tf
print(tf.test.is_gpu_available())


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False
